In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers, optimizers, datasets
import numpy as np

## Data 

**Data Load**

In [2]:
(images_train,labels_train), (images_test,labels_test) = datasets.mnist.load_data()
print('----------------train----------------')
print('images:',images_train.shape, images_train.min(), images_train.max()) # 모든 이미지는 픽셀 값으로 이루어짐
print('labels:',labels_train.shape)
print('----------------test----------------')
print('images:',images_test.shape, images_test.min(), images_test.max())
print('labels:',labels_test.shape)

----------------train----------------
images: (60000, 28, 28) 0 255
labels: (60000,)
----------------test----------------
images: (10000, 28, 28) 0 255
labels: (10000,)


**Normalizing**

In [13]:
# [0,255] to [0,1]

images_train = images_train/255.
images_test = images_test/255.

In [14]:
images_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

## Basic Model

* sequential로 모델 정의

In [15]:
# Flatten : 이미지 크기를 펴준다
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax') # 최종 10개 레이블이고 확률값을 얻기 위해 softmax
])

* label이 integer

In [16]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

* 학습

In [17]:
model.fit(images_train, labels_train, epochs=3, batch_size=64) # 60000/64=937.5

Epoch 1/3
938/938 [==============================] - 4s 3ms/step - loss: 2.3016 - accuracy: 0.1119
Epoch 2/3
938/938 [==============================] - 3s 3ms/step - loss: 2.3014 - accuracy: 0.1124
Epoch 3/3
938/938 [==============================] - 4s 4ms/step - loss: 2.3014 - accuracy: 0.1124


* 평가
  * verbose : 결과 출력 종류 (0,1,2)

In [22]:
test_loss, test_acc = model.evaluate(images_test,  labels_test, verbose=2)
test_loss, test_acc

313/313 - 1s - loss: 2.3011 - accuracy: 0.1135 - 562ms/epoch - 2ms/step


(2.301116943359375, 0.11349999904632568)

In [23]:
print('test accuracy:', test_acc)

test accuracy: 0.11349999904632568


### Dropout

In [34]:
class NeuralNet_dropout(Model):
    def __init__(self, hidden_1, hidden_2, num_classes):
        super(NeuralNet_dropout, self).__init__()
        self.flatten = layers.Flatten(input_shape=(28,28)) # flatten 해 주어야 모델 학습 가능
        self.fc1 = layers.Dense(hidden_1) # 첫 fc layer
        self.dropout1 = layers.Dropout(0.5) # Dropout(p)에서 p : 얼만큼 삭제할 것인가
        self.fc2 = layers.Dense(hidden_2)
        self.dropout2 = layers.Dropout(0.5)
        self.out = layers.Dense(num_classes)

    def call(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout1(x) # dropout : fc_layer와 활성화 함수 사이에
        x = tf.nn.relu(x)
        
        x = self.fc2(x)
        x = self.dropout2(x)
        x = tf.nn.relu(x)
        
        x = self.out(x)
        x = tf.nn.softmax(x)
        return x

In [35]:
hidden_1 = 128
hidden_2 = 256
num_classes = 10

model_dropout = NeuralNet_dropout(hidden_1, hidden_2, num_classes)

In [36]:
model_dropout.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
model_dropout.fit(images_train, labels_train, epochs=3, batch_size=64)

Epoch 1/3
938/938 [==============================] - 12s 10ms/step - loss: 2.3016 - accuracy: 0.1101
Epoch 2/3
938/938 [==============================] - 8s 8ms/step - loss: 2.3014 - accuracy: 0.1124
Epoch 3/3
938/938 [==============================] - 9s 10ms/step - loss: 2.3014 - accuracy: 0.1124


In [38]:
test_loss, test_acc = model_dropout.evaluate(images_test,  labels_test, verbose=2)

print('test accuracy:', test_acc)

313/313 - 1s - loss: 2.3010 - accuracy: 0.1135 - 1s/epoch - 3ms/step
test accuracy: 0.11349999904632568


### Batch Normalization

In [40]:
class NeuralNet_bn(Model):
    def __init__(self, hidden_1, hidden_2, num_classes):
        super(NeuralNet_bn, self).__init__()

        self.flatten = layers.Flatten(input_shape=(28,28))
        self.fc1 = layers.Dense(hidden_1)
        self.bn1 = layers.BatchNormalization()

        self.fc2 = layers.Dense(hidden_2)
        self.bn2 = layers.BatchNormalization()

        self.out = layers.Dense(num_classes)

    def call(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bn1(x)
        x = tf.nn.relu(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = tf.nn.relu(x)

        x = self.out(x)
        x = tf.nn.softmax(x)
        return x

* Hyperparameter : 인간이 수정할 수 있는 값
  * num_classes : mnist의 label이 10개라서 10이므로 이는 hyperparameter가 아님

In [41]:
hidden_1 = 128
hidden_2 = 256
num_classes = 10 # 데이터의 특성에 따라 정해짐

model_bn = NeuralNet_bn(hidden_1, hidden_2, num_classes)

In [42]:
model_bn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
model_bn.fit(images_train, labels_train, epochs=3, batch_size=64)

Epoch 1/3
938/938 [==============================] - 9s 8ms/step - loss: 2.3016 - accuracy: 0.1119
Epoch 2/3
938/938 [==============================] - 8s 8ms/step - loss: 2.3014 - accuracy: 0.1124
Epoch 3/3
938/938 [==============================] - 7s 8ms/step - loss: 2.3014 - accuracy: 0.1124


In [45]:
test_loss, test_acc = model_bn.evaluate(images_test,  labels_test, verbose=2)

print('test loss:', test_loss)
print('test accuracy:', test_acc)

313/313 - 1s - loss: 3.6181 - accuracy: 0.1135 - 1s/epoch - 4ms/step
test loss: 3.618062973022461
test accuracy: 0.11349999904632568
